In [6]:
import pandas as pd

In [9]:
DATA_PATH = "../data/CLEAN"

df_box_offices = pd.read_csv(DATA_PATH+"/Violent_Movies_final.tsv", sep="\t")

In [10]:
df_box_offices.head()

,Unnamed: 0,Wikipedia movie ID,Movie name,Release date,Box office revenue,Year,Week
0,0,975900,Ghosts of Mars,2001-08-24,14010832.0,2001,34
1,4,156558,Baby Boy,2001-06-27,29381649.0,2001,26
2,7,4951456,Kinjite: Forbidden Subjects,1989-02-03,3416846.0,1989,5
3,10,1369204,Juarez,1939-06-10,NaN,1939,23
4,13,2647998,The Human Tornado,1976-10-02,NaN,1976,40


In [11]:
df_box_offices.shape[0]

8261